<span style="color:DarkBlue;font-size:18px">Semesterabschließende schriftliche Ausarbeitung im Modul Machine Learning (SS2023)</span>

<span style="color:DarkBlue;font-size:32px">Preisvorhersage von Gebrauchtwagen </span>

<span style="color:DarkBlue;font-size:18px"> Bearbeitet von M.Sc. Onur Yilmaz</span>

### **Inhaltsverzeichnis**

#### **Einleitung**

#### **1. Datenaufbereitung**
#### **2. Explorative Datenanalyse**
#### **3. Feature Engineering**
#### **4. Modell Auswahl und Training**
#### **5. Modell Bewertung**
#### **6. Streamlit** 
#### **Literaturverzeichnis**




### **Einleitung**

In der vorliegenden semesterabschließenden Arbeit wird ein aufbereiteter Datensatz von 100.000 gebrauchten Autoanzeigen aus dem Vereinigten Königreich analysiert. Der Datensatz wurde von der Plattform Kaggle bezogen - https://www.kaggle.com/datasets/adityadesai13/used-car-dataset-ford-and-mercedes.

Das Hauptziel dieser Arbeit ist die Wahl und Implementierung eines geeigneten maschinellen Lernmodells zur Preisvorhersage von Gebrauchtwagen.

Die Arbeit gliedert sich in verschiedene Abschnitte, beginnend mit der Datenaufbereitung, gefolgt von der explorativen Datenanalyse und dem Feature Engineering. Anschließend wird das passende Modell ausgewählt und trainiert, bevor es bewertet wird. Zusätzlich wird die Implementierung in Streamlit durchgeführt, um eine interaktive Webanwendung zur Darstellung der Ergebnisse zu erstellen.

Der Datensatz ist in einzelne Dateien nach Autoherstellern unterteilt und enthält Informationen wie Preis, Getriebe, Kilometerstand, Kraftstoffart, Kfz-Steuer, Verbrauch in Meilen pro Gallone (mpg) und Motorgröße.

*Das gesamte Projekt kann aus dem folgenden Repository geklont werden: https://www.github.com/ONURYI*

### **1. Datenaufbereitung**

Die Datenaufbereitung ist oft ein entscheidender, aber sehr zeitaufwendiger Schritt, welches im Prozess des maschinellen Lernens vorab gemacht werden muss. Diese umfasst eine Reihe von Techniken zur Reinigung, Transformation und Organisation der Rohdaten in ein Format, das für die Analyse geeignet ist. Diese Phase legt das Fundament für die Modellbildung, indem sie sicherstellt, dass die Daten qualitativ hochwertig und relevant sind, und trägt so wesentlich zur Genauigkeit und Effizienz des endgültigen Modells bei [2].

Im Rahmen unseres Projektes beziehen wir die folgende CSV-Files (csv = comma seperated value):

- *audi.csv*
- *bmw.csv*
- *ford.csv*
- *hyundi.csv*
- *mercedes.csv*
- *skoda.csv*
- *toyota.csv*
- *vauxhall.csv*
- *vw.csv*


welche unter dem Ornder **/data** abgelegt sind und nach dem Aufbereitungs - und Bereinigungprozess dem Ordner **/data (clean)** als eine einzige CSV Datei hinzugefügt wird.

### **Literaturverzeichnis**

[1] https://www.kaggle.com/datasets/adityadesai13/used-car-dataset-ford-and-mercedes (01.08.2023)

[2] Müller, A. C., & Guido, S. (2021). Introduction to Machine Learning with Python: A Guide for Data Scientists (2nd ed.). O'Reilly Media